In [1]:
import pandas as pd
from numpy import nan
import string
import random
import pandas_tools as pt

In [2]:
# Set ourselves up to make a collection of "records",
# each of which contains data from a single observation in a nested dict format.
def rand_str(size=6, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for x in range(size))

rand_float = random.random
rand_int = lambda: random.randint(0,100)
# rand_value = lambda: random.choice([rand_str, rand_float, rand_int])()

In [3]:
def make_nasty_record(identifier):
    record = {
        'identifier': identifier,
        'category0': {
            'field0': rand_float(),
            'cat0_field0': rand_float(),
            'cat0_field1': rand_int(),
            'cat0_field2': {
                'cat0_field2_subfield0': rand_str(),
                'cat0_field2_subfield1': rand_float(),
                'cat0_field2_subfield2': {
                    'cat0_field2_subsubfield0': rand_int()
                },
            }
        },
        "category1": {
            'field0': rand_float(),
            'cat1_field0': {
                'cat1_field0_subfield0': {
                    'cat1_field0_subsubfield0': rand_str(),
                    'cat1_field0_subsubfield1': rand_float(),
                },
                'cat1_field0_subfield1': rand_str(),
            },
        },
    }
    return record

In [4]:
# Generate a list of records
n_records = 20
levels = ['category', 'field', 'subfield', 'subsubfield']
records = []
for _ in range(n_records):
    identifier = rand_str()
    records.append(make_nasty_record(identifier))

In [5]:
# the naive approach doesn't do what we want
df = pd.DataFrame.from_records(records).set_index('identifier')
df.head()
# could use some more structure...

,category0,category1
identifier,,
AY60M0,"{'field0': 0.7159498552077563, 'cat0_field0': ...","{'field0': 0.9383307483509256, 'cat1_field0': ..."
TU1S2F,"{'field0': 0.4123825135514271, 'cat0_field0': ...","{'field0': 0.9707514663021392, 'cat1_field0': ..."
NLEYKY,"{'field0': 0.5451587630399891, 'cat0_field0': ...","{'field0': 0.721439752200831, 'cat1_field0': {..."
LY7Z66,"{'field0': 0.03318838961716197, 'cat0_field0':...","{'field0': 0.9960775735273647, 'cat1_field0': ..."
Y1TSEJ,"{'field0': 0.015210012795852101, 'cat0_field0'...","{'field0': 0.6372873705461016, 'cat1_field0': ..."


In [6]:
# Create flat DataFrame
df = pt.df_from_records(records, index='identifier', levels=levels, flat=True)

In [7]:
# better, but still not very pretty...
df.head()

,category0.cat0_field0,category0.cat0_field1,category0.cat0_field2.cat0_field2_subfield0,category0.cat0_field2.cat0_field2_subfield1,category0.cat0_field2.cat0_field2_subfield2.cat0_field2_subsubfield0,category0.field0,category1.cat1_field0.cat1_field0_subfield0.cat1_field0_subsubfield0,category1.cat1_field0.cat1_field0_subfield0.cat1_field0_subsubfield1,category1.cat1_field0.cat1_field0_subfield1,category1.field0
identifier,,,,,,,,,,
AY60M0,0.385168,35,PLHXCX,0.255968,27,0.715950,OG8PPK,0.706275,479X8M,0.938331
TU1S2F,0.447821,65,IOP8CB,0.096920,47,0.412383,M3TYAP,0.292070,E12N75,0.970751
NLEYKY,0.325070,17,A2UISQ,0.132807,60,0.545159,M8SJP2,0.297235,OVVLT2,0.721440
LY7Z66,0.357351,42,KRT9BP,0.382050,40,0.033188,2CUUPM,0.714844,RWK4PP,0.996078
Y1TSEJ,0.417987,13,NODNOH,0.085444,70,0.015210,NTXVMI,0.856113,12L2O0,0.637287


In [8]:
# Create MultiIndexed DataFrame
df = pt.df_from_records(records, index='identifier', levels=levels, flat=False)

In [9]:
# Uneven nesting results in nan levels
df.head()

category      category0                                    \
field       cat0_field0 cat0_field1           cat0_field2   
subfield            NaN         NaN cat0_field2_subfield0   
subsubfield         NaN         NaN                   NaN   
identifier                                                  
AY60M0         0.385168          35                PLHXCX   
TU1S2F         0.447821          65                IOP8CB   
NLEYKY         0.325070          17                A2UISQ   
LY7Z66         0.357351          42                KRT9BP   
Y1TSEJ         0.417987          13                NODNOH   

category                                                              \
field                                                         field0   
subfield    cat0_field2_subfield1    cat0_field2_subfield2       NaN   
subsubfield                   NaN cat0_field2_subsubfield0       NaN   
identifier                                                             
AY60M0                   0.255968                       27  0.715950   
TU1S2F                   0.096920                       47  0.412383   
NLEYKY                   0.132807                       60  0.545159   
LY7Z66                   0.382050                       40  0.033188   
Y1TSEJ                   0.085444                       70  0.015210   

category                   category1                           \
field                    cat1_field0                            
subfield       cat1_field0_subfield0                            
subsubfield cat1_field0_subsubfield0 cat1_field0_subsubfield1   
identifier                                                      
AY60M0                        OG8PPK                 0.706275   
TU1S2F                        M3TYAP                 0.292070   
NLEYKY                        M8SJP2                 0.297235   
LY7Z66                        2CUUPM                 0.714844   
Y1TSEJ                        NTXVMI                 0.856113   

category                                     
field                                field0  
subfield    cat1_field0_subfield1       NaN  
subsubfield                   NaN       NaN  
identifier                                   
AY60M0                     479X8M  0.938331  
TU1S2F                     E12N75  0.970751  
NLEYKY                     OVVLT2  0.721440  
LY7Z66                     RWK4PP  0.996078  
Y1TSEJ                     12L2O0  0.637287

In [10]:
df.T.head()

identifier                                                              AY60M0  \
category  field       subfield              subsubfield                          
category0 cat0_field0 NaN                   NaN                       0.385168   
          cat0_field1 NaN                   NaN                             35   
          cat0_field2 cat0_field2_subfield0 NaN                         PLHXCX   
                      cat0_field2_subfield1 NaN                       0.255968   
                      cat0_field2_subfield2 cat0_field2_subsubfield0        27   

identifier                                                               TU1S2F  \
category  field       subfield              subsubfield                           
category0 cat0_field0 NaN                   NaN                        0.447821   
          cat0_field1 NaN                   NaN                              65   
          cat0_field2 cat0_field2_subfield0 NaN                          IOP8CB   
                      cat0_field2_subfield1 NaN                       0.0969197   
                      cat0_field2_subfield2 cat0_field2_subsubfield0         47   

identifier                                                              NLEYKY  \
category  field       subfield              subsubfield                          
category0 cat0_field0 NaN                   NaN                        0.32507   
          cat0_field1 NaN                   NaN                             17   
          cat0_field2 cat0_field2_subfield0 NaN                         A2UISQ   
                      cat0_field2_subfield1 NaN                       0.132807   
                      cat0_field2_subfield2 cat0_field2_subsubfield0        60   

identifier                                                              LY7Z66  \
category  field       subfield              subsubfield                          
category0 cat0_field0 NaN                   NaN                       0.357351   
          cat0_field1 NaN                   NaN                             42   
          cat0_field2 cat0_field2_subfield0 NaN                         KRT9BP   
                      cat0_field2_subfield1 NaN                        0.38205   
                      cat0_field2_subfield2 cat0_field2_subsubfield0        40   

identifier                                                               Y1TSEJ  \
category  field       subfield              subsubfield                           
category0 cat0_field0 NaN                   NaN                        0.417987   
          cat0_field1 NaN                   NaN                              13   
          cat0_field2 cat0_field2_subfield0 NaN                          NODNOH   
                      cat0_field2_subfield1 NaN                       0.0854444   
                      cat0_field2_subfield2 cat0_field2_subsubfield0         70   

identifier                                                              L54MFG  \
category  field       subfield              subsubfield                          
category0 cat0_field0 NaN                   NaN                       0.562536   
          cat0_field1 NaN                   NaN                             12   
          cat0_field2 cat0_field2_subfield0 NaN                         9XJ41G   
                      cat0_field2_subfield1 NaN                       0.647378   
                      cat0_field2_subfield2 cat0_field2_subsubfield0        26   

identifier                                                              HFNEXJ  \
category  field       subfield              subsubfield                          
category0 cat0_field0 NaN                   NaN                       0.428113   
          cat0_field1 NaN                   NaN                             42   
          cat0_field2 cat0_field2_subfield0 NaN                         8VKT2Z   
                      cat0_field2_subfield1 NaN                       0.176737   
                      cat0_field2_subfield2 

In [11]:
# direct access to a single columns, not that useful
print(df['category0', 'cat0_field1', nan, nan])

identifier
AY60M0    35
TU1S2F    65
NLEYKY    17
LY7Z66    42
Y1TSEJ    13
L54MFG    12
HFNEXJ    42
440EK2    15
2NMNJ6    71
TKXLH6    61
BPYUZL    51
NBXVTA     0
FE3Y1N    11
3TE8MU    28
UV8D63    88
HWXP1D     9
PXB6U5    89
ND1ZE2    37
R73KAR    95
STF76H    56
Name: (category0, cat0_field1, nan, nan), dtype: int64


In [12]:
print(df['category0', 'cat0_field1', nan, nan] is df[('category0', 'cat0_field1', nan, nan)])

True


In [13]:
# cross_section, a wrapper around pandas.DataFrame.xs, is a very useful function
pt.cross_section(df, category='category1').head()

field                    cat1_field0                           \
subfield       cat1_field0_subfield0                            
subsubfield cat1_field0_subsubfield0 cat1_field0_subsubfield1   
identifier                                                      
AY60M0                        OG8PPK                 0.706275   
TU1S2F                        M3TYAP                  0.29207   
NLEYKY                        M8SJP2                 0.297235   
LY7Z66                        2CUUPM                 0.714844   
Y1TSEJ                        NTXVMI                 0.856113   

field                                field0  
subfield    cat1_field0_subfield1       NaN  
subsubfield                   NaN       NaN  
identifier                                   
AY60M0                     479X8M  0.938331  
TU1S2F                     E12N75  0.970751  
NLEYKY                     OVVLT2   0.72144  
LY7Z66                     RWK4PP  0.996078  
Y1TSEJ                     12L2O0  0.637287

In [14]:
pt.cross_section(df, category='category1', drop_level=False).head()

category                   category1                           \
field                    cat1_field0                            
subfield       cat1_field0_subfield0                            
subsubfield cat1_field0_subsubfield0 cat1_field0_subsubfield1   
identifier                                                      
AY60M0                        OG8PPK                 0.706275   
TU1S2F                        M3TYAP                  0.29207   
NLEYKY                        M8SJP2                 0.297235   
LY7Z66                        2CUUPM                 0.714844   
Y1TSEJ                        NTXVMI                 0.856113   

category                                     
field                                field0  
subfield    cat1_field0_subfield1       NaN  
subsubfield                   NaN       NaN  
identifier                                   
AY60M0                     479X8M  0.938331  
TU1S2F                     E12N75  0.970751  
NLEYKY                     OVVLT2   0.72144  
LY7Z66                     RWK4PP  0.996078  
Y1TSEJ                     12L2O0  0.637287

In [15]:
pt.cross_section(df, category='category1', field=['field0', 'cat1_field0']).head()

category                   category1                           \
field                    cat1_field0                            
subfield       cat1_field0_subfield0                            
subsubfield cat1_field0_subsubfield0 cat1_field0_subsubfield1   
identifier                                                      
AY60M0                        OG8PPK                 0.706275   
TU1S2F                        M3TYAP                  0.29207   
NLEYKY                        M8SJP2                 0.297235   
LY7Z66                        2CUUPM                 0.714844   
Y1TSEJ                        NTXVMI                 0.856113   

category                                     
field                                field0  
subfield    cat1_field0_subfield1       NaN  
subsubfield                   NaN       NaN  
identifier                                   
AY60M0                     479X8M  0.938331  
TU1S2F                     E12N75  0.970751  
NLEYKY                     OVVLT2   0.72144  
LY7Z66                     RWK4PP  0.996078  
Y1TSEJ                     12L2O0  0.637287

In [16]:
# thanks to the handle_transpose decorator, operations work as expected on either df or df.T
pt.cross_section(df.T, category='category1', field=['field0', 'cat1_field0']).head()

identifier                                                              AY60M0  \
category  field       subfield              subsubfield                          
category1 cat1_field0 cat1_field0_subfield0 cat1_field0_subsubfield0    OG8PPK   
                                            cat1_field0_subsubfield1  0.706275   
                      cat1_field0_subfield1 NaN                         479X8M   
          field0      NaN                   NaN                       0.938331   

identifier                                                              TU1S2F  \
category  field       subfield              subsubfield                          
category1 cat1_field0 cat1_field0_subfield0 cat1_field0_subsubfield0    M3TYAP   
                                            cat1_field0_subsubfield1   0.29207   
                      cat1_field0_subfield1 NaN                         E12N75   
          field0      NaN                   NaN                       0.970751   

identifier                                                              NLEYKY  \
category  field       subfield              subsubfield                          
category1 cat1_field0 cat1_field0_subfield0 cat1_field0_subsubfield0    M8SJP2   
                                            cat1_field0_subsubfield1  0.297235   
                      cat1_field0_subfield1 NaN                         OVVLT2   
          field0      NaN                   NaN                        0.72144   

identifier                                                              LY7Z66  \
category  field       subfield              subsubfield                          
category1 cat1_field0 cat1_field0_subfield0 cat1_field0_subsubfield0    2CUUPM   
                                            cat1_field0_subsubfield1  0.714844   
                      cat1_field0_subfield1 NaN                         RWK4PP   
          field0      NaN                   NaN                       0.996078   

identifier                                                              Y1TSEJ  \
category  field       subfield              subsubfield                          
category1 cat1_field0 cat1_field0_subfield0 cat1_field0_subsubfield0    NTXVMI   
                                            cat1_field0_subsubfield1  0.856113   
                      cat1_field0_subfield1 NaN                         12L2O0   
          field0      NaN                   NaN                       0.637287   

identifier                                                              L54MFG  \
category  field       subfield              subsubfield                          
category1 cat1_field0 cat1_field0_subfield0 cat1_field0_subsubfield0    1UVA8D   
                                            cat1_field0_subsubfield1   0.67196   
                      cat1_field0_subfield1 NaN                         ZX50W0   
          field0      NaN                   NaN                       0.805208   

identifier                                                             HFNEXJ  \
category  field       subfield              subsubfield                         
category1 cat1_field0 cat1_field0_subfield0 cat1_field0_subsubfield0   2GG9MT   
                                            cat1_field0_subsubfield1  0.50569   
                      cat1_field0_subfield1 NaN                        L3LOLG   
          field0      NaN                   NaN                       0.80846   

identifier                                                              440EK2  \
category  field       subfield              subsubfield                          
category1 cat1_field0 cat1_field0_subfield0 cat1_field0_subsubfield0    V73MEA   
                                            cat1_field0_subsubfield1  0.112308   
                      cat1_field0_subfield1 NaN                         MWJJDJ   
          field0      NaN                   NaN                       0.303609   

identifier                                                    

In [17]:
pt.flatten(df).head()

,category0.cat0_field0,category0.cat0_field1,category0.cat0_field2.cat0_field2_subfield0,category0.cat0_field2.cat0_field2_subfield1,category0.cat0_field2.cat0_field2_subfield2.cat0_field2_subsubfield0,category0.field0,category1.cat1_field0.cat1_field0_subfield0.cat1_field0_subsubfield0,category1.cat1_field0.cat1_field0_subfield0.cat1_field0_subsubfield1,category1.cat1_field0.cat1_field0_subfield1,category1.field0
identifier,,,,,,,,,,
AY60M0,0.385168,35,PLHXCX,0.255968,27,0.715950,OG8PPK,0.706275,479X8M,0.938331
TU1S2F,0.447821,65,IOP8CB,0.096920,47,0.412383,M3TYAP,0.292070,E12N75,0.970751
NLEYKY,0.325070,17,A2UISQ,0.132807,60,0.545159,M8SJP2,0.297235,OVVLT2,0.721440
LY7Z66,0.357351,42,KRT9BP,0.382050,40,0.033188,2CUUPM,0.714844,RWK4PP,0.996078
Y1TSEJ,0.417987,13,NODNOH,0.085444,70,0.015210,NTXVMI,0.856113,12L2O0,0.637287
